# Rough Work

1. ## Reading receipts

In [117]:
import pandas as pd
import numpy as np
import pytesseract
import cv2
from PIL import Image
import os

2. ## Manipulating data

In [118]:

file_path = f"{os.getcwd()}/test_files/receipt1.jpeg"

image = cv2.imread(file_path)


In [119]:
import tempfile
import cv2  
import numpy as np
from PIL import Image

def deskew(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    coords = np.column_stack(np.where(binary > 0))
    contour_points = [tuple(coord) for coord in coords]
    rect = cv2.minAreaRect(np.array(contour_points))
    angle = rect[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

def set_image_dpi(image, output_file_path):
    im = Image.fromarray(image)
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.ANTIALIAS)
    im_resized.save(output_file_path, dpi=(300, 300))
    return output_file_path

def remove_noise(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)

def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

def proceaa_image(image):
    deskewed_image = deskew(image)
    noise_removed_image = remove_noise(deskewed_image)
    grayscale_image = get_grayscale(noise_removed_image)
    thresholded_image = thresholding(grayscale_image)
    return thresholded_image
# Read the image
file_path = f"{os.getcwd()}/test_files/receipt1.jpeg"
image = cv2.imread(file_path)

# Process the image
processed_image = proceaa_image(image)

# Save the processed image
output_file_path = f"{os.getcwd()}/test_files/output1.jpeg"
cv2.imwrite(output_file_path, processed_image)


True

# <ins>Image Processing</ins>

In [120]:
import cv2
import numpy as np
import os
import PIL as image

In [121]:
file = f"{os.getcwd()}/test_files/receipt3.jpeg"
output_file_path = f"{os.getcwd()}/test_files/output.jpeg"
img = cv2.imread(file)
pilimg = Image.open(file)


def cv_save(file_path,image):
    file = f"{os.getcwd()}/{file_path}"
    return cv2.imwrite(file, image)


## 1. Inverted Images


In [122]:
inverted_img = cv2.bitwise_not(img)
cv2.imwrite(output_file_path,img=inverted_img)

'''Thi is not necessary in tesseract 4'''

'Thi is not necessary in tesseract 4'

## 2. Rescaling


## 3. Binarization


In [136]:
def grey_scale(image):
    """
    Convert the image to greyscale.
    """
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

img = cv2.imread(file)


gray_image = grey_scale(inverted_img)



cv2.imwrite(f"{os.getcwd()}/test_files/grey_scale.jpeg",gray_image)

True

In [137]:
thresh,im_bw = cv2.threshold(gray_image,200,230,cv2.THRESH_BINARY)

cv2.imwrite(f'{os.getcwd()}/test_files/bw.jpg', im_bw)

True

## 4. Noise Removal


In [125]:
import cv2
import numpy as np
def noise_removal(imag):
    # noise removal
    kernel = np.ones((1,1),np.uint8)
    image = cv2.dilate(imag,kernel,iterations=1)
    kernel = np.ones((1,1),np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return image

In [126]:
no_noise = noise_removal(im_bw)
cv_save('/test_files/noise.jpg',no_noise)

True

In [135]:
image = Image.open(f"{os.getcwd()}/test_files/bw.jpg")
text = pytesseract.image_to_string(image)
print(text)

 

”



## 5. Dilation and Erosion


In [128]:
def thin_font(image):
    image = cv2.bitwise_not(image)
    kernel = np.ones((2, 2), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return image


In [129]:
erroded = thin_font(no_noise)
cv_save("test_files/erroded.jpeg",erroded)

True

In [130]:
def thick_font(image):
    image = cv2.bitwise_not(image)
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return image

In [131]:
dilated = thick_font(no_noise)
cv_save("test_files/dilated.jpeg",dilated)



True

## 6. Rotation / Deskewing

In [132]:
def getSkewAngle(cvImage) -> float:
    # Prep image, copy, convert to gray scale, blur, and threshold
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    for c in contours:
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        cv2.rectangle(newImage,(x,y),(x+w,y+h),(0,255,0),2)

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    print (len(contours))
    minAreaRect = cv2.minAreaRect(largestContour)
    cv2.imwrite("temp/boxes.jpg", newImage)
    # Determine the angle. Convert it to the value that was originally used to obtain skewed image
    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90 + angle
    return -1.0 * angle
# Rotate the image around its center
def rotateImage(cvImage, angle: float):
    newImage = cvImage.copy()
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage



In [133]:
def deskew(cvImage):
    angle = getSkewAngle(cvImage)
    return rotateImage(cvImage, -1.0 * angle)

In [134]:
fixed = deskew(img)
cv_save("test_files/deskewed.jpg",fixed)


144


True

## 7. Removing Borders

## 8. Missing Borders
